In [1]:
import mitiq

mitiq.SUPPORTED_PROGRAM_TYPES.keys()

['braket', 'cirq', 'pennylane', 'pyquil', 'qibo', 'qiskit']

In [2]:
frontend = "cirq"

In [3]:
from mitiq import benchmarks

circuits = benchmarks.generate_rb_circuits(
  n_qubits=1, num_cliffords=2, return_type = frontend, trials=3,
)
circuit = circuits[0]
print(circuits[0])
print(circuits[1])
print(circuits[2])

0: ───Y^-0.5───X^-0.5───Y^0.5───Y^0.5───X^0.5───Y^0.5───X───
0: ───Y^0.5───X^0.5───Y^-0.5───X^-0.5───Y^0.5───(Y-X^0.5)───
0: ───Y^-0.5───I───Y^0.5───I───I───


In [4]:
import numpy as np
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq

def execute(circuit, noise_level=0.01):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    executed with depolarizing noise.
    """
    # Replace with code based on your frontend and backend.
    mitiq_circuit, _ = convert_to_mitiq(circuit)
    noisy_circuit = mitiq_circuit.with_noise(depolarize(p=noise_level))
    rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    return rho[0, 0].real

In [5]:
# Compute the expectation value of the |0><0| observable.
noisy_value = execute(circuit)
ideal_value = execute(circuit, noise_level=0.0)
print(f"Error without mitigation: {abs(ideal_value - noisy_value) :.5f}")

Error without mitigation: 0.04484


In [7]:
from mitiq import zne
print(linear_fac.get_scale_factors())

mitigated_result = zne.execute_with_zne(circuits, execute, factory=linear_fac)

[]
shivansh
[0.9607930183410645, 0.9228377342224121, 0.8868370056152344, 0.98039710521698, 0.9566540718078613, 0.9434801340103149, 0.9902970194816589, 0.9761162400245667, 0.9725918769836426]
<class 'list'>


/Users/Shared/Drive/Qiskit/mitiq/mitiq/zne/inference.py:178: ExtrapolationWarning: The extrapolation fit may be ill-conditioned. Likely, more data points are necessary to fit the parameters of the model.
  opt_params, params_cov = np.polyfit(
/Users/Shared/Drive/Qiskit/mitiq/mitiq/zne/inference.py:182: ExtrapolationWarning: The extrapolation fit may be ill-conditioned. Likely, more data points are necessary to fit the parameters of the model.
  opt_params = np.polyfit(scale_factors, exp_values, deg, w=weights)


In [8]:
print(mitigated_result)

0.9160624991011783


In [9]:
print(f"Error with mitigation (ZNE): {abs(ideal_value - mitigated_result):.{3}}")

Error with mitigation (ZNE): 0.0839


In [10]:
from mitiq.zne.scaling import fold_gates_at_random

folded = fold_gates_at_random(circuit, scale_factor=2.)
print("Folded circuit:", folded, sep="\n")

Folded circuit:
0: ───Y^-0.5───Y^0.5───Y^-0.5───X^-0.5───Y^0.5───Y^0.5───X^0.5───Y^0.5───Y^-0.5───Y^0.5───X───X───X───


In [11]:
from mitiq.zne.inference import LinearFactory

linear_fac = LinearFactory(scale_factors=[1.0, 2.0, 3.0])
linear_fac.get_scale_factors()

[]

In [12]:
from mitiq import zne

mitigated_result = zne.execute_with_zne(
     circuit, execute, factory=linear_fac, scale_noise=fold_gates_at_random,
)

In [13]:
print(f"Error with mitigation (ZNE): {abs(ideal_value - mitigated_result):.{3}}")

Error with mitigation (ZNE): 0.0046


# Two stage application of ZNE

In [14]:
from mitiq.zne import scaled_circuits, combine_results
from mitiq.zne.scaling import fold_gates_at_random

scale_factors = [1.0, 2.0, 3.0]

folded_circuits = scaled_circuits(
    circuit=circuit,
    scale_factors=[1.0, 2.0, 3.0],
    scale_method=fold_gates_at_random,
)

folded_circuits

[0: ───Y^-0.5───X^-0.5───Y^0.5───Y^0.5───X^0.5───Y^0.5───X───,
 0: ───Y^-0.5───X^-0.5───X^0.5───X^-0.5───Y^0.5───Y^-0.5───Y^0.5───Y^0.5───X^0.5───Y^0.5───X───X───X───,
 0: ───Y^-0.5───Y^0.5───Y^-0.5───X^-0.5───X^0.5───X^-0.5───Y^0.5───Y^-0.5───Y^0.5───Y^0.5───Y^-0.5───Y^0.5───X^0.5───X^-0.5───X^0.5───Y^0.5───Y^-0.5───Y^0.5───X───X───X───]

In [15]:
results = [execute(circuit) for circuit in folded_circuits]

print(results)

[0.95515895, 0.9199386, 0.8771806]


In [16]:
from mitiq.zne.inference import RichardsonFactory

extrapolation_method = RichardsonFactory(scale_factors=scale_factors).extrapolate
two_stage_zne_result = combine_results(
    scale_factors, results, extrapolation_method
)

print(f'Unmitigated value: {execute(circuit)}')
print(f'Mitigated value: {two_stage_zne_result}')

Unmitigated value: 0.9551589488983154
Mitigated value: 0.9828415513038642
